In [ ]:
import requests
from bs4 import BeautifulSoup 
import re
import time

pref_code = [46] #都道府県コード
block_code = [1008] #地域コード
place_name = ["小田原"]

In [ ]:
# DBファイルを保存するためのファイルパス

import sqlite3

# Github
path = '/content/'

# DBファイル名
db_name = 'github.sqlite'

# DBに接続する（指定したDBファイル存在しない場合は，新規に作成される）
con = sqlite3.connect(path + db_name)
cur = conn.cursor()

# テーブルが存在しない場合は作成
cur.execute('''CREATE TABLE IF NOT EXISTS weather
                (date TEXT, precipitation FLOAT, temp_avg FLOAT, temp_high FLOAT, temp_low FLOAT, humidity_avg FLOAT, humidity_min FLOAT, sunshine FLOAT)''')

# DBへの接続を閉じる
con.commit()
con.close()

In [ ]:
url = "https://www.data.jma.go.jp/obd/stats/etrn/view/daily_a1.php?prec_no=%s&block_no%s&year=%s&month=%s&day=17&view=p1"

# 取ったデータをfloat型に変える(データが取れなかった場合、気象庁は"/"を埋め込んでいるため0に変える)
def str2float(str):
  try:
    return float(str)
  except:
    return 0.0